In [20]:
from dotenv import load_dotenv

load_dotenv()

True

In [37]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Dict, Any
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
tool = TavilySearchResults(max_results=2)

In [6]:
# Agent state to maintain memory
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [7]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [48]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}
        

In [40]:
from contextlib import ExitStack

stack = ExitStack()
memory = stack.enter_context(SqliteSaver.from_conn_string(":memory:"))

In [49]:
import os
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-5-nano", api_key=os.environ.get("OPENAI_API_KEY"))
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [51]:
messages = [HumanMessage(content="What is the weather in London?")]

In [43]:
thread = {"configurable": {"thread_id": "1"}}

In [50]:
for event in abot.graph.stream({"messages": messages}, config=thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RFARkcs2WYbF3rPiWhh2XFwf', 'function': {'arguments': '{"query":"San Francisco weather current"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 414, 'prompt_tokens': 268, 'total_tokens': 682, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C2xahhrIaUz8eIzhBKdPysAnWbFLX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--49fbcd22-88cc-495a-824d-7480c872286b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather current'}, 'id': 'call_RFARkcs2WYbF3rPiWhh2XFwf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 26

In [52]:
messages = [HumanMessage(content="what about in London?")]
thread = {'configurable': {"thread_id": "1"}}

for event in abot.graph.stream({"messages": messages}, config=thread):
    for value in event.values():
        print(value)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hqgFmIbR3tWzoK3CepQZycNG', 'function': {'arguments': '{"query": "London current weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_ih9agr1YXlb0e1oHR69bQURs', 'function': {'arguments': '{"query": "London weather now"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_jBKDmRB29fWBBgr7ecWf8TlP', 'function': {'arguments': '{"query": "London UK weather current conditions"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 343, 'prompt_tokens': 5840, 'total_tokens': 6183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C2xfZAMnP

In [54]:
messages = [HumanMessage(content="which is warmer?")]
thread = {'configurable': {"thread_id": "1"}}

for event in abot.graph.stream({"messages": messages}, config=thread):
    for value in event.values():
        print(value)

{'messages': [AIMessage(content='Short answer: London is warmer right now.\n\n- London: about 17–19°C (63–66°F)\n- San Francisco: about 14°C (57°F)\n\nDifference: London is roughly 3–5°C (6–9°F) warmer at the moment. If you want, I can pull exact current temps for a specific London area or show the next 24-hour forecast.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 730, 'prompt_tokens': 9596, 'total_tokens': 10326, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 9344}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C2xj4WZbDC1blHAryWoCOkIkV3ukM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3421c841-ef30-40c7-a7c3-c11997430ac8-0', usage_metadata={'input_tokens': 9596, 'output_tokens': 730, 'total_tokens': 10326, '

# Async streaming of token

In [ ]:
# Note - don;t use mem saveer in prod, this was only to get the tutorial to run, use sqlite or posgres in prod.
from langgraph.checkpoint.memory import MemorySaver

memory =  MemorySaver()
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

async def ask_bot(question: str):
    messages = [HumanMessage(content=question)]
    thread = {"configurable": {"thread_id": "4"}}

    async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            # Empty content means tool call so don't print
            if content:
                print(content, end="|")

await ask_bot("Yes give me the weather right now")

I| can| pull| that| up|.| Which| city| (|and| country|)| would| you| like| the| current| weather| for|?| And| do| you| prefer| Celsius| or| Fahrenheit|?| If| you| tell| me| a| location|,| I|’ll| fetch| the| latest| conditions|.|

In [69]:
await ask_bot("Colindale london england")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Colindale London England current weather'}, 'id': 'call_uydCyMAtTTVF9H6bZxuhaagh', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Colindale current weather Colindale NW9'}, 'id': 'call_IkNdJkXhbIaHEkOPleeIj0OZ', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'BBC Weather Colindale'}, 'id': 'call_TiKEvIyMq1BdPur8QET2e0M7', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Colindale current weather now temperature'}, 'id': 'call_vJjnAllZw1EyQzWAs7l2dZJ6', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Colindale current weather temperature now Colindale NW9'}, 'id': 'call_5VtjoFprw9AsxBEjnoTLBtR9', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Colindale London current 

CancelledError: 